In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

from config import db_password

import time


In [2]:
#  Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    
    # STEP 1 MAKE A EMPTY DICT TO HOLD ALTERNATIVE LANGUAGE TITLES
    alt_titles = {} 
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        
        # STEP 2-A CHECK IF THE CURRENT KEY EXISTS IN MOVIE OBJECT
        # STEP 2-B IF YES, REMOVE KEY-VALUE PAIR; ADD TO ALT TITLES DICT.
        if key in movie:  
            alt_titles[key] = movie[key]  
            movie.pop(key)
            
        # STEP 3 LOOP THRU EVERY KEY, ADD ALT TITLES DICT TO MOVIE OBJECT            
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles  

    # MERGE AND UPDATE TO CONSOLIDATE COLUMN NAMES
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie
  

In [3]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load():
    
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    #kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)
    #ratings = pd.read_csv(ratings_file) 
    kaggle_metadata = pd.read_csv(f'{file_dir}/movies_metadata.csv', low_memory=False)
    ratings = pd.read_csv(f'{file_dir}/ratings.csv')
    
    # Open and read the Wikipedia data JSON file.
    with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
    #with open(wiki_file, mode='r') as file: 
        wiki_movies_raw = json.load(file)

    # Write a list comprehension to filter out TV shows.
    wiki_movies = [movie for movie in wiki_movies_raw if ('No. of episodes' not in movie)]

    
    
    # Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies = [clean_movie(movie) for movie in wiki_movies]

    # Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_movies)
    sorted(wiki_movies_df.columns.tolist())

    # Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
        
    except:
        print("Error found during extraction for IMDb ID")  
        
    #  Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
    
    
    # Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df['Box office'].dropna()
    
    # Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

    # Write a regular expression to match the six elements of "form_one" of the box office data.
    # PART I: START WITH A DOLLAR SIGN
    # PART II: ADD AN ARBITRARY (BUT NON-ZERO) NUMBER OF DIGITS
    # PART III: ADD AN OPTIONAL DECIMANL POINT
    # PART IV: ADD AN ARBITRARY (BUT POSSIBLY ZERO) NUMBER OF MORE DIGITS
    # PART V: ADD A SPACE (OR MAYBE MORE THAN ONE)
    # PART VI: ADD THE WORD "MILLION" OR "BILLION"
    form_one = r'\$\d+\.?\d*\s*[mb]illion'
    
    # Write a regular expression to match the three elements of "form_two" of the box office data.
    # PART I: START WITH A DOLLAR SIGN
    # PART II: ADD A GROUP OF ONE TO THREE DIGIS
    # PART III: MATCH AT LEAST ONE GROUP STARTING WITH A COMMA AND FOLLOWED BY EXACTLY 3 DIGITS
    form_two = r'\$\d{1,3}(?:,\d{3})+'
    box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()

    # Add the parse_dollars function.
    def parse_dollars(s):
        # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

        # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a million
            value = float(s) * 10**6

            # return value
            return value

        # if input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a billion
            value = float(s) * 10**9

            # return value
            return value

        # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

            # remove dollar sign and commas
            s = re.sub('\$|,','', s)

            # convert to float
            value = float(s)

            # return value
            return value

        # otherwise, return NaN
        else:
            return np.nan
   
        # Clean the box office column in the wiki_movies_df DataFrame.
        # 1) IDENTIFY VALUES THAT HAVE SPACES IN BETWEEN THE $ SIGN AND THE NUMBER
        form_one = r'\$\s*\d+\.?\d*\s*[mb]illion'
        form_two = r'\$\s*\d{1,3}(?:,\d{3})+'

        # 2) IDENTIFY VALUES THAT USE A PERIOD, NOT A COMMA AS A 'THOUSANDS' SEPARATOR
        form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'

        # 3) IDENTIFY VALUES THAT DISPLAY A RANGE (INSTEAD OF A SINGLE AMOUNT)
        box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

        # 4) IDENTIFY SPELLING ERRORS OF 'MILLION' (SPELLED AS 'MILLON')
        form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'

        box_office.str.extract(f'({form_one}|{form_two})')



        # Clean the budget column in the wiki_movies_df DataFrame.
        # STEP 1: CREATE A BUDGET VARIABLE
        budget = wiki_movies_df['Budget'].dropna()

        # STEP 2: CONVERT ANY LISTS TO STRINGS
        budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)

        # STEP 3: REMOVE ANY VALUES BETWEEN $ AND A HYPHEN (FOR BUDGETS GIVEN IN RANGES)
        budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

        # COMPARE VALUES IN FORM ONE & TWO OF BUDGET DATA (TO SEE WHICH VALUES DO NOT MATCH)
        matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE, na=False)
        matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE, na=False)
        budget[~matches_form_one & ~matches_form_two]

        # MODULE 8.3.11 REMOVE CITATION REFERENCES (SQUARE BRACKETS) IN BUDGET DATA
        budget = budget.str.replace(r'\[\d+\]\s*', '', regex=True)
        budget[~matches_form_one & ~matches_form_two]

        # MODULE 8.3.11  PARSE THE BUDGET DATA TO A NUMERICAL FORM (REPLACED 'box_office' WITH 'budget')
        wiki_movies_df['Budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

        # DROP THE BUDGET COLUMN AS WE NO LONGER NEED IT
        wiki_movies_df.drop('Budget', axis=1, inplace=True)



        # Clean the release date column in the wiki_movies_df DataFrame.
        # STEP 1: CREATE VARIABLE TO HOLD NON-NULL VALUES; CONVERT LISTS TO STRINGS
        release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

        # STEP 2: PARSE THRU DIFFERENT RELEASE DATE DISPLAY FORMATS
        date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
        date_form_two = r'\d{4}.[01]\d.[0123]\d'
        date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
        date_form_four = r'\d{4}'

        # STEP 3: EXTRACT AND CONVERT THE RELEASE DATE VALUES
        release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)

        # USE PANDAS to_datetime() METHOD  & infer_datetime_format OPTION TO ALIGN RELEASE DATE FORMAT
        wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)



        # Clean the running time column in the wiki_movies_df DataFrame.
        # STEP 1: CREATE A VARIABLE TO HOLD NON-NULL VALUES; CONVERT LISTS TO STRINGS
        running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

        # STEP 2: PARSE THRU DIFFERENT RUNNING TIME DISPLAY FORMATS
        running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False).sum()

        # STEP 3-A: INSPECT THE REMAINING RUNNING TIME DISPLAY FORMATS
        running_time[running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False) != True]

        # STEP 3-B: NARROW SEARCH TO INSPECT RUNNING TIME DISPLAY; (INSTEAD OF 'MINUTES', SEARCH STRINGS THAT START WITH 'M')
        running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False).sum()

        # STEP 3-C: INSPECT REMAINING RUNNING TIME DISPLAY FORMATS
        running_time[running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False) != True]

        # STEP 4-A: EXTRACT AND CONVERT RUNNING TIME VALUES
        running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')

        # STEP 4-B: CONVERT STRINGS TO NUMERIC VALUES
        # USE PANDAS to_numeric METHOD; THEN fillna() FUNCTION TO CHANGE ALL 'NaN's TO ZEROS
        running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)

        # STEP 4-C: CONVERT VALUES WITH 'hour(s) + minutes' INTO ONLY 'minutes'
        wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)

        # DROP RUNNING TIME COLUMN AS WE NO LONGER NEED IT
        wiki_movies_df.drop('Running time', axis=1, inplace=True)


        # 2. Clean the Kaggle metadata.
        kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')
        kaggle_metadata['video'] == 'True'
        kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'

        # MODULE 8.3.12 CONVERT DATA TYPES FOR NUMERIC COLUMNS: 'budget', 'id' AND 'popularity'
        # USE PANDAS to-numeric METHOD
        kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
        kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
        kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')

        kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])


        # 3. Merged the two DataFrames into the movies DataFrame.
        # MODULE 8.4.1 MERGE wiki_movies_df AND kaggle_metadata; IDENTIFY REDUNDANT COLUMNS
        movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])


        # 4. Drop unnecessary columns from the merged DataFrame.
        # drop the title_wiki, release_date_wiki, Language, and Production company(s) columns.
        movies_df.drop(columns=['title_wiki','release_date_wiki','Language','Production company(s)'], inplace=True)


        # 5. Add in the function to fill in the missing Kaggle data.
        def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
            df[kaggle_column] = df.apply(
            lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
            , axis=1)
            df.drop(columns=wiki_column, inplace=True)


        # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.
        fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
        fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
        fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')


        # 7. Filter the movies DataFrame for specific columns.
        # MODULE 8.4.1 RE-ORDER THE COLUMNS TO MAKE DATASET EASIER TO READ
        movies_df = movies_df.loc[:, ['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
                           'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
                           'genres','original_language','overview','spoken_languages','Country',
                           'production_companies','production_countries','Distributor',
                           'Producer(s)','Director','Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on'
                          ]]

        # 8. Rename the columns in the movies DataFrame.
        movies_df.rename({'id':'kaggle_id',
                      'title_kaggle':'title',
                      'url':'wikipedia_url',
                      'budget_kaggle':'budget',
                      'release_date_kaggle':'release_date',
                      'Country':'country',
                      'Distributor':'distributor',
                      'Producer(s)':'producers',
                      'Director':'director',
                      'Starring':'starring',
                      'Cinematography':'cinematography',
                      'Editor(s)':'editors',
                      'Writer(s)':'writers',
                      'Composer(s)':'composers',
                      'Based on':'based_on'
                     }, axis='columns', inplace=True)


        # 9. Transform and merge the ratings DataFrame.

        # MODULE 8.4.2 TRANSFORM AND MERGE RATING DATA
        # STEP 1: GET THE COUNT FOR EACH GROUP: 'movieid' AND 'rating' COLUMNS
        rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count()

        # STEP 2-A: RE-NAME 'userid' COLUMN TO 'count'
        rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                    .rename({'userId':'count'}, axis=1)

        # STEP 2-B: PIVOT DATA; MAKE 'movieId' THE INDEX, COLUMNS WILL BE RATING VALUES AND ROWS WILL BE COUNTS FOR EACH RATING VALUE
        rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                    .rename({'userId':'count'}, axis=1) \
                    .pivot(index='movieId',columns='rating', values='count')

        # STEP 2-C: RE-NAME THE COLUMNS; PREPEND (ADD AT THE BEGINNING OF) 'rating_' TO EACH COLUMN
        rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]

        # MERGE THE RATING COUNTS INTO 'movies_df'; LEFT MERGE
        movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')

        # FILL IN MISSING RATING LEVEL VALUES WITH A ZERO
        movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)

        return wiki_movies_df, movies_with_ratings_df, movies_df


In [4]:
# 10. Create the path to your file directory and variables for the three files.
file_dir ='C:/Users/takyu/CLASSWORK/Movies-ETL/resources'

# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'

# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'

# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'


In [5]:
# 11. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file


In [ ]:
wiki_movies_df.head()


In [ ]:
# 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df.head()


In [ ]:
# 14. Check the movies_with_ratings_df DataFrame.
movies_with_ratings_df.head()


In [ ]:
# 15. Check the movies_df DataFrame. 
movies_df.head()
